In [ ]:
%matplotlib ipympl
import numpy as np
import scipy.signal
import matplotlib.pylab as plt
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import YouTubeVideo

# Detectando ondas gravitacionales con un matched filter

## Introducción

Una onda gravitacional es una perturbación en el espacio-tiempo causada por la interacción de dos cuerpos super masivos. Su existencia fue predicha por Einstein en base a su teoría de relatividad general.

Sin embargo, no fue hasta 2016 cuando el Laser Interferometer Gravitational-Wave Observatory (LIGO) observó la primera onda gravitacional

Las ondas graviatacionales nos permiten estudiar objetos astronómicos que son dificiles o imposibles de observar por otros medios, por ejemplo los agujeros negros

In [ ]:
YouTubeVideo('p43sb92YOww')

Las ondas gravitacionales deben viajar distancias enormes para llegar a la Tierra. Por ende lo que recibimos es una señal muy débil y altamente contaminada por ruido como muestra la siguiente figura

<img src="https://www.researchgate.net/publication/320975513/figure/fig1/AS:613937242452040@1523385453807/Sample-signal-injected-into-real-LIGO-noise-The-red-time-series-is-an-example-of-the.png" width="500">

Podemos utilizar un filtro FIR para encontrar esta señal en el ruido. La técnica usada por la colaboración LIGO para detectar ondas gravitacionales es el [matched filter](https://www.ligo.org/science/Publication-GW150914CBC/index.php)

## Matched filter

EL match filter es un filtro muy sencillo pero muy robusto. Se usa para detectar una señal $s$ a partir de datos $x$ contaminados con ruido aditivo $\epsilon$

$$
x[n] = s[n] + \epsilon[n],
$$

definamos ahora un filtro FIR

$$
y[n] = (h * x)[n] = \sum_{k} h[n-k] x[k],
$$

Siguiendo el supuesto de ruido aditivo tendríamos que

$$
y[n] = y_s[n] + y_\epsilon[n]
$$

Luego podemos escribir la razón señal a ruido como

$$
\text{SNR} = \frac{|y_s|^2}{\mathbb{E}[y_\epsilon^2]} = \frac{(h * s)^T (h * s)}{\sigma^2 |h|^2}
$$

Si [maximizamos el numerador](https://en.wikipedia.org/wiki/Matched_filter#Derivation_via_Lagrangian) bajo la restricción de que el filtro es ortogonal ($|h|^2=I$) se obtiene el filtro óptimo

$$
h = \frac{s}{|s|}
$$

Es decir que el filtro óptimo para encontrar $s$ está basado en $s$. 


## Un modelo de onda gravitacional

Para encontrar ondas gravitacionales con el matched filter necesitamos un modelo matemático que represente $s$. Este modelo viene dado por los modelos físicos (teóricos) del fenómeno.

Este modelo tiene parámetros que al variarlos producen una familia de plantillas (templates) 

En este caso utilizaremos una simplificación de dicho modelo con dos parámetros: El largo temporal y la frecuencia

In [ ]:
def gravitational_wave(length, freq, Fs=100):
    time = np.arange(0, length*1.1, step=1/Fs)
    signal = scipy.signal.chirp(time, freq, length, 5*freq, method='hyperbolic')
    time = time - length
    signal[time <= 0] *= (time[time<=0]+length)*0.01
    signal[time > 0] *= -(time[time>0]-length*0.1)*0.1
    return time, signal

def update_plot(length, freq):
    ax.cla()
    time, template = gravitational_wave(length, freq, Fs=100)
    ax.plot(time, template)
    
fig, ax = plt.subplots(figsize=(6, 3), tight_layout=True)
interact(update_plot, length=IntSlider(min=10, max=500, step=10, value=50),
         freq=FloatSlider(min=0.01, max=0.2, step=1e-2, value=0.1));

## Buscando señales en el ruido utilizando plantillas


Simulemos los datos que recibe el sensor de LIGO. En estos datos con muy baja SNR hay escondida una onda gravitacional

In [ ]:
np.random.seed(1234)

true_length, true_freq, Fs = 50, 0.1234, 100
_, template = gravitational_wave(true_length, true_freq, Fs)

s = np.zeros(shape=(100*Fs,))
s[1234:1234+len(template)] = template
x =  s + 0.5*np.random.randn(*s.shape)

fig, ax = plt.subplots(2, figsize=(6, 4), tight_layout=True)
ax[0].plot(np.arange(0, 100, step=1/Fs), x)
freqs, times, Sxx = scipy.signal.spectrogram(x, fs=Fs, nperseg=512)
ax[1].pcolormesh(times, freqs, np.log10(Sxx+1e-4), shading='gouraud', cmap=plt.cm.Reds)
#ax[1].set_ylim(0, 5)

Para encontrarle generaremos una serie de plantillas y las usaremos como filtro para convolucionar la señal

Para cada filtro $h$ registraremos el valor máximo de $y = h*x$ y también el retardo donde ocurre este máximo

El más grande entre todos los máximos será el mejor template o plantilla

In [ ]:
lengths = np.arange(20, 500, step=10)
freqs = np.arange(0.01, 0.2, step=1e-2)
best_value = np.zeros(shape=(len(lengths), len(freqs)))
best_lag = np.zeros_like(best_value)

for i, length in enumerate(lengths):
    for j, freq in enumerate(freqs):
        _, h = gravitational_wave(length, freq, Fs)
        y = scipy.signal.correlate(x, h, mode='valid')/np.sqrt(np.sum(h**2))
        best_lag[i, j] = np.argmax(y)
        best_value[i, j] = np.max(y)
        
fig, ax = plt.subplots(figsize=(6 ,4), tight_layout=True)
ax.pcolormesh(freqs, lengths, best_value, shading='auto', cmap=plt.cm.Blues)

idx = np.unravel_index(np.argmax(best_value), best_value.shape)

print(lengths[idx[0]], freqs[idx[1]])
print(best_lag[idx[0], idx[1]])

Visualicemos el mejor template encontrado (naranjo) y el resultado de la convolución. 

En azul se muestra la señal deseada. En la práctica no tenemos esta señal pero aquí la podemos visualizar como referencia.

¿Cómo cambia el resultado si cambiamos los arreglos `lengths` y `freqs`?

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True)


for _, h in [gravitational_wave(true_length, true_freq, Fs), 
          gravitational_wave(lengths[idx[0]], freqs[idx[1]], Fs)
          ]:
    
    y = scipy.signal.correlate(x, h, mode='valid')/np.sqrt(np.sum(h**2))
    ax[0].plot(y)    
    ax[1].plot(h)
    
ax[0].set_title('Convolución (y)')
ax[1].set_title('Template [h]');

## Comentarios y discusión

- El matched filter requiere que el ruido sea aditivo
- El template debe ser idealmente idéntico a la señal que se busca
- El método de fuerza bruta es muy costoso. Modelos más flexibles con más parámetros se vuelven rapidamente infactibles. Una alternativa es usar técnicas de optimización
- Referencia: https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.120.141103